**Emotion Recognition Model**
**Training and Testing**




All these imports are needed, so please install all these libraries

**Note:** Please change all the directories as per your system

In [ ]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
from tensorflow import keras
from tensorflow.keras import layers

These are the sex classes of emotions, each number representing one emotion:


1.   '0' represents 'angry' 
2.   '1' represents 'disgust'
3.   '2' represents 'fear'
4.   '3' represents 'happy'
5.   '4' represents 'neutral'
6.   '5' represents 'sad'
7.   '6' represents 'surprised'

The folders containing the testing and training images, should have subfolders numbered as above. For example, all 'angry' face images should present in folder names '0'.





In [ ]:
Classes = ["0", "1", "2", "3", "4", "5", "6"]

MobileNetv2 requires image size of 224*224


In [ ]:
img_size=224

This function is needed to create testing and training data from the images

In [ ]:

def create_data(data_directory):
    data = []
    count = 0
    file_no = 1
    for category in Classes:
        path = os.path.join(data_directory, category)
        class_num = Classes.index(category)
        for img in  os.listdir(path):
            count+=1
            print(count)
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (img_size, img_size)) ##Bilinear interpolation
                data.append([new_array, class_num])
                
            except Exception as e:
                pass

    return data

This part creates the training data which includes the image preprocessing

In [ ]:
data_directory = "/content/drive/MyDrive/archive/Training"
training_data = create_data(data_directory)
random.shuffle(training_Data)
X = []
y = []
for features, label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, img_size, img_size, 3)


This is to save the dataset as the files can get big

In [ ]:
np.save('/content/drive/MyDrive/archive/train_X.npy', X)
np.save('/content/drive/MyDrive/archive/train_y.npy', y)

This part downloads the pretrained MobileNetV2 model. More layers are added on this model so that it can detect our seven classes of emotions. For loss funtion "sparse_categorical_crossentropy" is used and  "adam" is used as optimizer.

In [ ]:
model = tf.keras.applications.MobileNetV2() 
final_output = layers.Dense(128)(base_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7, activation='softmax')(final_output)
new_model = keras.Model(inputs = base_input, outputs = final_output)
new_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics = ["accuracy"])

This  part is used to train the model

In [ ]:
new_model.fit(np.load('/content/drive/MyDrive/archive/train_X.npy'), np.load('/content/drive/MyDrive/archive/train_y.npy'), batch_size = 1, epochs = 10)

This is used to save the trained model in '.h5' format.

In [ ]:
new_model.save('<path to save model>')

This part loads the saved model

In [ ]:
new_model = tf.keras.models.load_model('<saved model path>')

This part is used to generate the testing dataset

In [ ]:
data_directory = "/content/drive/MyDrive/archive/Testing"
testing_data = create_data(data_directory)
X = []
y = []
for features, label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, img_size, img_size, 3)


This parts does testing on the training data set.

In [ ]:
correct_map = [0, 0, 0, 0, 0, 0, 0]
wrong_map = [0, 0, 0, 0, 0, 0, 0]

count = 0
for img_index in range(len(X)):
  Predictions = new_model.predict(np.expand_dims(X[img_index], axis=0))
  count +=1
  print(count)
  if np.argmax(Predictions) == 0:
    if y[img_index] == 0:
      correct_map[0] += 1
    else:
      wrong_map[y[img_index]] += 1

  elif np.argmax(Predictions) == 1:
    if y[img_index] == 1:
      correct_map[1] += 1
    else:
      wrong_map[y[img_index]] += 1    
        
  elif np.argmax(Predictions) == 2:
    if y[img_index] == 2:
      correct_map[2] += 1
    else:
      wrong_map[y[img_index]] += 1
        
        
  elif np.argmax(Predictions) == 3:
    if y[img_index] == 3:
      correct_map[3] += 1
    else:
      wrong_map[y[img_index]] += 1
        
        
  elif np.argmax(Predictions) == 4:
    if y[img_index] == 4:
      correct_map[4] += 1
    else:
      wrong_map[y[img_index]] += 1
        
    
  elif np.argmax(Predictions) == 5:
    if y[img_index] == 5:
      correct_map[5] += 1
    else:
      wrong_map[y[img_index]] += 1
        
    
  else :
    if y[img_index] == 6:
      correct_map[6] += 1
    else:
      wrong_map[y[img_index]] += 1